In [ ]:
import sqlite3
from datetime import datetime

class WordList:
    def __init__(self, db_name="language_learning.db"):
        self.db_name = db_name
        self.conn = sqlite3.connect(self.db_name)
        self.cursor = self.conn.cursor()
        self._create_table()
        self.table_name = 'words'

    def _create_table(self,name='words'):
        self.table_name = name
        query = f""" 
        CREATE TABLE IF NOT EXISTS {self.table_name} (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            word TEXT NOT NULL,
            translation TEXT NOT NULL,
            translation2 TEXT NOT NULL,
            notes TEXT,
            difficulty INTEGER DEFAULT 1,
            date_added TEXT DEFAULT CURRENT_TIMESTAMP
        );
        """
        self.cursor.execute(query)
        self.conn.commit()
    def add_word(self, word, translation, translation2='', notes=None):
        # check if the word is already in the list
        query = f"""
        SELECT id, word, translation 
        FROM {self.table_name} 
        WHERE word = ?;
        """
        self.cursor.execute(query, (word,))
        result =  self.cursor.fetchall()
        if len(result) == 0:
            query = f""" 
            INSERT INTO {self.table_name} (word, translation, translation2, notes) 
            VALUES (?, ?, ?, ?);
            """
            self.cursor.execute(query, (word, translation, translation2, notes))
            self.conn.commit()
        else:
            print(f"This word: {word} is already in the list.")
    def add_word_input(self):
        word = input("Word in dutch:")
        translation = input("Translation in english:")
        translation2 = input("Translation in russian:")
        self.add_word(word, translation, translation2)
        
    def get_all_words(self):
        self.cursor.execute(f"SELECT id, word, translation, translation2 FROM {self.table_name};")
        return self.cursor.fetchall()
    
    def delete_word(self, word_id):
        self.cursor.execute(f"DELETE FROM {self.table_name} WHERE id = ?;", (word_id,))
        self.conn.commit()

    def empty_the_list(self):
        full_list = self.get_all_words()
        for i, item in enumerate(full_list):
            self.delete_word(item[0])
        
    
    def search_word(self, keyword):
        query = f"""
        SELECT id, word, translation, translation2
        FROM {self.table_name} 
        WHERE word LIKE ? OR translation LIKE ? OR translation2 LIKE ?;
        """
        self.cursor.execute(query, (keyword, keyword, keyword))
        return self.cursor.fetchall()
    
    def ingest_list(self, words_list):
        for item in words_list:
            self.add_word(item[0],item[1],item[2])


In [ ]:
class Learning:
    def __int__(self, words_list):
        self.status = 0
    # def CardsLearning(self):


In [ ]:
a = WordList()

In [137]:
a.add_word_input()

In [138]:
a.empty_the_list()


In [78]:
b = a.search_word('de auto')
b

[]

In [140]:
c = a.get_all_words()
c

[(122, 'fel', 'bright', 'яркий'),
 (123,
  'telkens',
  'each time, every time, again and again',
  'каждый раз, снова и снова'),
 (124, 'het suizen', 'whistling', 'свист, шум, шипение'),
 (125, 'het geruis', 'rustling', 'шорох'),
 (126, 'vergissen', 'to be mistaken', 'ошибаться'),
 (127, 'het zuiden', 'south', 'юг'),
 (128, 'bewijzen', 'to prove', 'доказывать'),
 (129, 'de nagel', 'nail', 'ноготь'),
 (130, 'het tikken', 'ticking', 'стук, тиканье'),
 (131, 'het geweld', 'violence', 'насилие'),
 (132, 'verblinden', 'to blind', 'ослеплять'),
 (133, 'vertragen', 'to slow down', 'замедлять'),
 (134, 'opschrikken', 'to startle', 'вздрагивать'),
 (135, 'huiveren', 'to shiver', 'дрожать'),
 (136, 'uitdoven', 'to extinguish', 'гасить'),
 (137, 'aangeven', 'to indicate, to report', 'указать, сообщать'),
 (138, 'benaderen', 'to approach', 'приближаться'),
 (139, 'ongepast', 'inappropriate', 'неуместный'),
 (140,
  'duiden',
  'to point at, to indicate, to interpret',
  'указывать, интерпретирова

In [47]:
from IPython.display import Markdown

# Beautiful printing of the text
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


In [81]:
import glob
import os
import re
import pathlib
import time
import textwrap
import google.generativeai as genai
with open("var.txt","r") as f:
    var = f.read()
genai.configure(api_key=var)
model = genai.GenerativeModel('models/gemini-2.0-flash')

In [ ]:
to_markdown(chat_completion.candidates[0].content.parts[0].text)

In [131]:
with open('input3.txt', 'r') as f:
    data = f.read()


In [ ]:

prompt_text = """
Here is a list of dutch words and short phrases. Some of them are translated to english and russian, some only to english. 
Add missing translations of dutch words to english or to russian columns and return it in json format. The json format should be the following:
{'althans': {'english': 'at least', 'russian': 'по крайней мере'},
'overleggen': {'english': 'to discuss', 'russian': 'обсуждать'},
'het hoorcollege': {'english': 'lecture', 'russian': 'лекция'},
}
The words: """ 
chat_completion = model.generate_content(prompt_text + data)



In [133]:
import json
raw_string = chat_completion.candidates[0].content.parts[0].text

# 1. Clean up the Markdown code block wrappers
clean_json = raw_string.strip().removeprefix('```json').removesuffix('```').strip()

# 2. Parse it with json.loads()
data = json.loads(clean_json)

# 3. Convert it to a 2D lis
words_list = []
for dutch_word in data:
    dutch = dutch_word
    try:
      english = data[dutch_word]['english']
    except KeyError:
       english = ''
    try: 
      russian = data[dutch_word]['russian']
    except KeyError:
       russian = ''
    words_list.append([dutch, english, russian]) 



In [139]:
a.ingest_list(words_list=words_list)